In [9]:
!pip install lifelines tableone

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 10.4 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=877a91d8aa0602e943932d76413b25ba76d999b7ea7fe38f52ddca9134a46ca9
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma


In [10]:
# ==============================================================================
# === SETUP: Configuration & Dependencies =====================================
# ==============================================================================
print("--- Initializing Setup ---")

# --- Configuration ---
COHORT_CSV_PATH = '/content/drive/MyDrive/DukeDatathon_2025_Team3/Brian/files/df_cohort_v0.csv'
PATIENTS_CSV_PATH = '/content/drive/MyDrive/DukeDatathon_2025_Team3/Brian/files/patients.csv'
RESULTS_OUTPUT_DIR = '/content/drive/MyDrive/DukeDatathon_2025_Team3/AnalysisResults/'

# --- Dependencies ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pickle
import os
import traceback
import json
import matplotlib.pyplot as plt # For KM plots
import seaborn as sns # For KM plots styling

try:
    from tableone import TableOne
    print("Successfully imported tableone.")
except ImportError:
    print("INFO: Package 'tableone' not found. Install with: !pip install tableone")
    TableOne = None

try:
    from lifelines import KaplanMeierFitter, CoxPHFitter
    from lifelines.statistics import logrank_test
    print("Successfully imported lifelines.")
except ImportError:
    print("INFO: Package 'lifelines' not found. Install with: !pip install lifelines")
    KaplanMeierFitter, CoxPHFitter, logrank_test = None, None, None # Set to None

# Google Drive specific
from google.colab import drive

# --- Mount Google Drive ---
print("\nAttempting to mount Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"ERROR: Error mounting Google Drive: {e}")
    raise SystemExit("Stopping: Cannot access Google Drive.")

# --- Create Output Directory ---
print(f"\nEnsuring results directory exists: {RESULTS_OUTPUT_DIR}")
try:
    os.makedirs(RESULTS_OUTPUT_DIR, exist_ok=True)
    print("Results directory ready.")
except Exception as e:
    print(f"ERROR: Error creating output directory: {e}")
    print("Warning: Proceeding, but saving results might fail.")

# ==============================================================================
# === PART 1: DATA LOADING, PREPARATION & OUTCOME CREATION ====================
# ==============================================================================
print("\n" + "="*60)
print("=== PART 1: DATA LOADING, PREPARATION & OUTCOME CREATION ===")
print("="*60)
try:
    # Load the initial cohort created by Brian
    print(f"Loading cohort data from: {COHORT_CSV_PATH}")
    df_cohort = pd.read_csv(COHORT_CSV_PATH)
    print(f"  Initial cohort shape: {df_cohort.shape}")

    # Load the patients table to get Date of Death (dod)
    print(f"Loading patients data from: {PATIENTS_CSV_PATH}")
    df_patients = pd.read_csv(PATIENTS_CSV_PATH)

    # Convert relevant time columns to datetime objects
    print("\nConverting time columns...")
    time_cols_cohort = ['intime', 'deathtime']
    for col in time_cols_cohort:
        if col in df_cohort.columns:
             df_cohort[col] = pd.to_datetime(df_cohort[col], errors='coerce')
             print(f"  Converted '{col}' in cohort.")

    # --- Merge Date of Death (dod) ---
    print("\nMerging Date of Death (dod)...")
    df_cohort = pd.merge(df_cohort, df_patients[['subject_id', 'dod']], on='subject_id', how='left')
    df_cohort['dod'] = pd.to_datetime(df_cohort['dod'], errors='coerce')
    print(f"  Cohort shape after merging 'dod': {df_cohort.shape}")
    print("  Merged 'dod' successfully.")

    # --- Create the Accurate 30-Day Mortality Outcome ---
    print("\nCreating 'mortality_30day' outcome...")
    time_diff_days = (df_cohort['dod'] - df_cohort['intime']).dt.days
    df_cohort['mortality_30day'] = 0
    df_cohort.loc[(time_diff_days >= 0) & (time_diff_days <= 30), 'mortality_30day'] = 1
    print("  Set mortality based on 'dod' within 30 days.")
    if 'hospital_expire_flag' in df_cohort.columns:
        initial_deaths = df_cohort['mortality_30day'].sum()
        df_cohort.loc[df_cohort['hospital_expire_flag'] == 1, 'mortality_30day'] = 1
        final_deaths = df_cohort['mortality_30day'].sum()
        print(f"  Refined mortality using 'hospital_expire_flag' (added {final_deaths - initial_deaths} potential events).")
    else: print("  Warning: 'hospital_expire_flag' not found.")
    print("  'mortality_30day' column created.")
    print("-" * 30)

    # --- Prepare Features: Renaming, Cleaning, Imputation ---
    print("\nPreparing Features...")
    if 'race' in df_cohort.columns and 'ethnicity' not in df_cohort.columns:
        df_cohort.rename(columns={'race': 'ethnicity'}, inplace=True)
        print("  Renamed 'race' to 'ethnicity'.")
        race_col = 'ethnicity'
    elif 'ethnicity' in df_cohort.columns: race_col = 'ethnicity'
    elif 'race' in df_cohort.columns: race_col = 'race'
    else: race_col = None

    sdoh_cols_to_clean = [col for col in [race_col, 'insurance', 'language'] if col is not None and col in df_cohort.columns]
    print(f"  Cleaning SDoH columns: {sdoh_cols_to_clean}")
    for col in sdoh_cols_to_clean:
        df_cohort[col] = df_cohort[col].fillna('UNKNOWN').astype(str)
        value_counts = df_cohort[col].value_counts(normalize=True)
        rare_cats = value_counts[value_counts < 0.01].index
        rare_cats = [cat for cat in rare_cats if cat not in ['UNKNOWN', 'OTHER']]
        if rare_cats: df_cohort[col] = df_cohort[col].replace(rare_cats, 'OTHER')

    predictor_cols_to_impute = ['age', 'sofa', 'charlson_comorbidity_index']
    print(f"\n  Imputing missing values in predictors: {predictor_cols_to_impute}")
    for col in predictor_cols_to_impute:
        if col in df_cohort.columns:
            if df_cohort[col].isnull().any():
                num_missing = df_cohort[col].isnull().sum()
                median_val = df_cohort[col].median()
                df_cohort[col].fillna(median_val, inplace=True)
                print(f"    Imputed {num_missing} missing '{col}' with median ({median_val}).")
        else: print(f"    Warning: Predictor '{col}' not found.")
    print("\nData Preparation Complete.")

except Exception as e:
    print(f"ERROR during Data Loading/Preparation: {e}")
    traceback.print_exc()
    raise SystemExit("Stopping due to critical error in Data Loading/Preparation.")


# ==============================================================================
# === PART 2: EXPLORATORY DATA ANALYSIS (EDA) & TABLEONE ======================
# ==============================================================================
print("\n" + "="*60)
print("=== PART 2: EXPLORATORY DATA ANALYSIS (EDA) & TABLEONE ===")
print("="*60)
eda_results = {} # Initialize dictionary to store summaries
try:
    print("Calculating Overall EDA summaries...")
    eda_results['total_stays'] = int(df_cohort['stay_id'].nunique())
    eda_results['total_patients'] = int(df_cohort['subject_id'].nunique())
    eda_results['overall_mortality_rate_30day'] = float(df_cohort['mortality_30day'].mean())
    print(f"  Overall Metrics: Stays={eda_results['total_stays']}, Patients={eda_results['total_patients']}, Mortality={eda_results['overall_mortality_rate_30day']:.3f}")

    if TableOne:
        print("\nGenerating TableOne summaries...")
        tableone_cols = ['age', 'gender', race_col, 'insurance', 'language', 'sofa', 'charlson_comorbidity_index']
        tableone_categorical = ['gender', race_col, 'insurance', 'language']
        tableone_nonnormal = ['age', 'sofa', 'charlson_comorbidity_index']
        tableone_cols = [col for col in tableone_cols if col in df_cohort.columns]
        tableone_categorical = [col for col in tableone_categorical if col in df_cohort.columns]
        tableone_nonnormal = [col for col in tableone_nonnormal if col in df_cohort.columns]

        try:
            print("  Calculating Overall Table 1...")
            table1_overall = TableOne(df_cohort, columns=tableone_cols, categorical=tableone_categorical, nonnormal=tableone_nonnormal, pval=False, htest_name=False)
            table1_overall_file = os.path.join(RESULTS_OUTPUT_DIR, 'table1_overall.csv')
            table1_overall.to_csv(table1_overall_file)
            print(f"  Overall Table 1 saved to {table1_overall_file}")
        except Exception as e: print(f"  ERROR generating Overall Table 1: {e}")

        try:
            print("\n  Calculating Table 1 stratified by 30-Day Mortality...")
            table1_strat_mort = TableOne(df_cohort, columns=tableone_cols, categorical=tableone_categorical, nonnormal=tableone_nonnormal, groupby='mortality_30day', pval=True, htest_name=True)
            table1_strat_mort_file = os.path.join(RESULTS_OUTPUT_DIR, 'table1_stratified_mortality.csv')
            table1_strat_mort.to_csv(table1_strat_mort_file)
            print(f"  Stratified Table 1 (Mortality) saved to {table1_strat_mort_file}")
        except Exception as e: print(f"  ERROR generating Mortality-Stratified Table 1: {e}")

        if race_col and race_col in df_cohort.columns:
            try:
                print(f"\n  Calculating Table 1 stratified by {race_col}...")
                table1_strat_sdoh = TableOne(df_cohort, columns=tableone_cols, categorical=tableone_categorical, nonnormal=tableone_nonnormal, groupby=race_col, pval=False, htest_name=False)
                table1_strat_sdoh_file = os.path.join(RESULTS_OUTPUT_DIR, f'table1_stratified_{race_col}.csv')
                table1_strat_sdoh.to_csv(table1_strat_sdoh_file)
                print(f"  Stratified Table 1 ({race_col}) saved to {table1_strat_sdoh_file}")
            except Exception as e: print(f"  ERROR generating {race_col}-Stratified Table 1: {e}")
        else: print(f"  Skipping Table 1 stratified by ethnicity/race as column '{race_col}' was not found/valid.")
    else:
        print("\nPackage 'tableone' not installed. Skipping detailed Table 1 generation.")
        print("  Calculating basic summaries instead...")
        numeric_summary = df_cohort[[col for col in ['age', 'sofa', 'charlson_comorbidity_index'] if col in df_cohort.columns]].describe().to_dict()
        eda_results['numeric_summary'] = numeric_summary
        categorical_counts = {}
        for col in ['gender'] + sdoh_cols_to_clean:
            if col in df_cohort.columns: categorical_counts[col] = df_cohort[col].value_counts().to_dict()
        eda_results['categorical_counts'] = categorical_counts
        print("  Basic summaries calculated.")

    eda_results_file = os.path.join(RESULTS_OUTPUT_DIR, 'eda_results.pkl')
    with open(eda_results_file, 'wb') as f: pickle.dump(eda_results, f)
    print(f"\nBasic EDA results dictionary saved to: {eda_results_file}")
except Exception as e:
    print(f"An error occurred during EDA: {e}")
    traceback.print_exc()


# ==============================================================================
# === PART 3: EQUITY ANALYSIS (Stratified Logistic Regression) ================
# ==============================================================================
print("\n" + "="*60)
print("=== PART 3: EQUITY ANALYSIS (Stratified Logistic Regression) ===")
print("="*60)
target = 'mortality_30day'
numeric_features = ['age', 'sofa', 'charlson_comorbidity_index']
categorical_features = ['gender']
sdoh_factors = [col for col in [race_col, 'insurance', 'language'] if col is not None and col in df_cohort.columns]

analysis_needed_cols = [target] + numeric_features + categorical_features + sdoh_factors
missing_analysis_cols = [col for col in analysis_needed_cols if col not in df_cohort.columns]
if missing_analysis_cols:
    print(f"FATAL ERROR: Cannot proceed. Missing columns: {missing_analysis_cols}")
    raise SystemExit("Stopping due to missing columns for analysis.")

print(f"Starting Stratified Analysis using SDoH factors: {sdoh_factors}")
results_auc = {}
results_roc = {}
try:
    for factor in sdoh_factors:
        print(f"\n--- Analyzing Factor: {factor} ---")
        results_auc[factor] = {}
        results_roc[factor] = {}
        categories = df_cohort[factor].unique()
        for category in categories:
            if pd.isna(category): continue
            print(f"  Processing Category: {category}")
            df_group = df_cohort[df_cohort[factor] == category].copy()
            if len(df_group) < 50 or df_group[target].nunique() < 2:
                print(f"    Skipping category '{category}' (data={len(df_group)}, outcomes={df_group[target].nunique()})")
                continue
            current_numeric_features = [f for f in numeric_features if f in df_group.columns]
            current_categorical_features = [f for f in categorical_features if f in df_group.columns]
            features_to_use = current_numeric_features + current_categorical_features
            X = df_group[features_to_use]
            y = df_group[target]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
            transformers = []
            if current_numeric_features: transformers.append(('num', StandardScaler(), current_numeric_features))
            if current_categorical_features: transformers.append(('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), current_categorical_features))
            if not transformers: continue
            preprocessor = ColumnTransformer(transformers=transformers, remainder='passthrough')
            model_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', LogisticRegression(solver='liblinear', random_state=42, class_weight='balanced'))])
            try:
                model_pipeline.fit(X_train, y_train)
                y_pred_proba = model_pipeline.predict_proba(X_test)[:, 1]
                auc = roc_auc_score(y_test, y_pred_proba)
                results_auc[factor][category] = auc
                fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
                results_roc[factor][category] = {'fpr': fpr.tolist(), 'tpr': tpr.tolist(), 'auc': auc}
                print(f"    Category: {category} | Test AUC: {auc:.4f}")
            except Exception as model_e: print(f"    ERROR processing category '{category}': {model_e}")
    print("\n--- Equity Analysis Finished ---")
    print("\nAUC Results Summary:")
    print(json.dumps(results_auc, indent=2, default=str))
    print("\nSaving analysis results...")
    auc_results_file = os.path.join(RESULTS_OUTPUT_DIR, 'equity_auc_results.pkl')
    roc_results_file = os.path.join(RESULTS_OUTPUT_DIR, 'equity_roc_results.pkl')
    with open(auc_results_file, 'wb') as f: pickle.dump(results_auc, f)
    print(f"  AUC results saved successfully to: {auc_results_file}")
    with open(roc_results_file, 'wb') as f: pickle.dump(results_roc, f)
    print(f"  ROC results saved successfully to: {roc_results_file}")
except Exception as e:
    print(f"An critical error occurred during the Equity Analysis phase: {e}")
    traceback.print_exc()


# ==============================================================================
# === PART 4: SURVIVAL ANALYSIS PREPARATION ===================================
# ==============================================================================
print("\n" + "="*60)
print("=== PART 4: SURVIVAL ANALYSIS PREPARATION ===")
print("="*60)

# This part calculates the duration variable needed for KM and Cox
try:
    print("Calculating time-to-event ('duration_30day') for survival analysis...")
    # Ensure intime and dod are datetime objects (might be redundant but safe)
    df_cohort['intime'] = pd.to_datetime(df_cohort['intime'], errors='coerce')
    df_cohort['dod'] = pd.to_datetime(df_cohort['dod'], errors='coerce')

    # Time difference in days (using total_seconds for precision)
    time_diff = (df_cohort['dod'] - df_cohort['intime']).dt.total_seconds() / (60*60*24)

    # Calculate duration: time to death if < 30 days, otherwise 30 days (censoring time)
    # Censor at 30 days if dod is null OR if death occurred after 30 days
    df_cohort['duration_30day'] = np.where(
        pd.notna(time_diff) & (time_diff <= 30) & (time_diff >= 0),
        time_diff, # Use actual time difference if death within 30 days
        30.0       # Censor at 30 days otherwise
    )
    # Ensure duration is non-negative
    df_cohort['duration_30day'] = df_cohort['duration_30day'].clip(lower=0.001) # Use small positive floor for stability

    print("  Created 'duration_30day' column.")
    print("  Sample of duration and mortality:")
    print(df_cohort[['intime', 'dod', 'mortality_30day', 'duration_30day']].head())
    print("\n  Summary statistics for 'duration_30day':")
    print(df_cohort['duration_30day'].describe())

except Exception as e:
    print(f"ERROR during Survival Analysis Preparation: {e}")
    traceback.print_exc()
    print("Warning: Survival prep failed. Subsequent survival analyses might fail.")

# ==============================================================================
# === PART 5: SURVIVAL ANALYSIS (Kaplan-Meier & Cox) ==========================
# ==============================================================================
print("\n" + "="*60)
print("=== PART 5: SURVIVAL ANALYSIS (Kaplan-Meier & Cox) ===")
print("="*60)

# Check if lifelines is available and data is ready
if KaplanMeierFitter is None or CoxPHFitter is None:
    print("Skipping Survival Analysis because 'lifelines' package is not installed.")
elif 'duration_30day' not in df_cohort.columns or 'mortality_30day' not in df_cohort.columns:
     print("Skipping Survival Analysis because 'duration_30day' or 'mortality_30day' column is missing.")
else:
    # --- Kaplan-Meier Analysis ---
    print("\n--- Kaplan-Meier Survival Analysis ---")
    km_results = {} # To store KM plots/data if needed for Streamlit (optional)

    # Define duration and event columns
    duration_col = 'duration_30day'
    event_col = 'mortality_30day'

    # Plot KM curves stratified by SDoH factors
    for factor in sdoh_factors:
        print(f"\n  Generating KM plot stratified by '{factor}'...")
        plt.figure(figsize=(10, 6))
        ax = plt.gca() # Get current axes

        categories = sorted(df_cohort[factor].unique()) # Sort categories for consistent legend order

        # Store results for pairwise logrank tests
        logrank_results = {}

        for i, category in enumerate(categories):
            # Filter data for the category
            df_group = df_cohort[df_cohort[factor] == category]

            if len(df_group) < 20: # Need sufficient data for KM curve
                 print(f"    Skipping KM for category '{category}' in factor '{factor}' (too few samples: {len(df_group)})")
                 continue

            # Fit Kaplan-Meier estimator
            kmf = KaplanMeierFitter()
            kmf.fit(df_group[duration_col], event_observed=df_group[event_col], label=f"{category} (n={len(df_group)})")

            # Plot the curve
            kmf.plot_survival_function(ax=ax, ci_show=False) # ci_show=False for cleaner multi-group plot

            # Compare with the reference group (e.g., the first category) using logrank test
            if i > 0:
                 ref_group = df_cohort[df_cohort[factor] == categories[0]]
                 try:
                     results = logrank_test(ref_group[duration_col], df_group[duration_col],
                                            ref_group[event_col], df_group[event_col])
                     logrank_results[f"{categories[0]} vs {category}"] = results.p_value
                     print(f"    Log-rank test ({categories[0]} vs {category}): p-value = {results.p_value:.4f}")
                 except Exception as lr_e:
                      print(f"    Could not perform logrank test for {category}: {lr_e}")


        # Customize and save the plot
        plt.title(f'30-Day Survival Stratified by {factor.replace("_", " ").title()}')
        plt.xlabel('Days Since ICU Admission')
        plt.ylabel('Survival Probability')
        plt.ylim(0, 1)
        plt.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.legend(title=factor.replace("_", " ").title())
        plt.tight_layout()
        km_plot_file = os.path.join(RESULTS_OUTPUT_DIR, f'km_plot_{factor}.png')
        try:
            plt.savefig(km_plot_file, dpi=150)
            print(f"  KM plot saved to {km_plot_file}")
            plt.close() # Close the plot to avoid displaying it inline here
            km_results[f'km_plot_{factor}'] = km_plot_file # Store path for potential Streamlit use
            km_results[f'logrank_{factor}'] = logrank_results # Store p-values
        except Exception as save_e:
            print(f"  ERROR saving KM plot for {factor}: {save_e}")
            plt.close()


    # --- Cox Proportional Hazards Analysis ---
    print("\n--- Cox Proportional Hazards Analysis ---")
    cox_results = {} # To store Cox model summaries

    # Prepare data for Cox model - requires dummy variables for categoricals
    print("  Preparing data for Cox model (dummy variables)...")
    try:
        # Select columns needed
        cox_cols = [duration_col, event_col, 'age', 'sofa', 'charlson_comorbidity_index', 'gender'] + sdoh_factors
        df_cox = df_cohort[cox_cols].copy()

        # Create dummy variables for ALL categorical predictors
        # drop_first=True to avoid multicollinearity
        categorical_for_cox = ['gender'] + sdoh_factors
        df_cox_dummies = pd.get_dummies(df_cox, columns=categorical_for_cox, drop_first=True, dummy_na=False) # dummy_na=False ignores NaNs created by cleaning
        print(f"  Data prepared for Cox model with {df_cox_dummies.shape[1]} columns.")
        # print("  Columns for Cox:", df_cox_dummies.columns.tolist()) # Uncomment to check columns

        # 1. Main Effects Model
        print("\n  Fitting Cox Main Effects Model...")
        cph_main = CoxPHFitter(penalizer=0.01) # Small penalizer for stability
        # Ensure all column names are strings and valid identifiers if necessary
        df_cox_dummies.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df_cox_dummies.columns]
        # Re-define duration and event cols in case names were changed
        duration_col_cox = "".join (c if c.isalnum() else "_" for c in str(duration_col))
        event_col_cox = "".join (c if c.isalnum() else "_" for c in str(event_col))
        cph_main.fit(df_cox_dummies, duration_col=duration_col_cox, event_col=event_col_cox)
        print("  Main Effects Model Summary:")
        cph_main.print_summary(decimals=3, style='ascii')
        cox_results['main_summary_df'] = cph_main.summary

        # Save main model summary
        main_cox_file = os.path.join(RESULTS_OUTPUT_DIR, 'cox_model_main_summary.csv')
        cph_main.summary.to_csv(main_cox_file)
        print(f"  Main Cox model summary saved to {main_cox_file}")

        # 2. Interaction Model (SOFA * SDoH) - More Complex
        print("\n  Fitting Cox Interaction Model (SOFA * SDoH)...")
        # Need to define interaction terms carefully
        # We focus on SOFA interactions with the *reference* dummy variables
        df_interaction = df_cox_dummies.copy()
        sofa_col_cox = "".join (c if c.isalnum() else "_" for c in str('sofa')) # Get cleaned sofa column name

        interaction_terms = []
        base_formula_cols = [col for col in df_interaction.columns if col not in [duration_col_cox, event_col_cox]]

        # Create interaction terms manually
        for factor in sdoh_factors:
             # Get dummy columns related to this factor (excluding the dropped reference level)
             dummy_cols = [col for col in df_interaction.columns if col.startswith(factor + "_")]
             for dummy_col in dummy_cols:
                  interaction_name = f"{sofa_col_cox}_x_{dummy_col}"
                  df_interaction[interaction_name] = df_interaction[sofa_col_cox] * df_interaction[dummy_col]
                  interaction_terms.append(interaction_name)

        if interaction_terms:
            print(f"  Created {len(interaction_terms)} interaction terms for SOFA.")
            # Fit model including interaction terms
            cph_interact = CoxPHFitter(penalizer=0.01)
            formula_cols = base_formula_cols + interaction_terms
            # Ensure no duplicates
            formula_cols = list(dict.fromkeys(formula_cols))

            cph_interact.fit(df_interaction[[duration_col_cox, event_col_cox] + formula_cols], duration_col=duration_col_cox, event_col=event_col_cox)
            print("\n  Interaction Model Summary:")
            cph_interact.print_summary(decimals=3, style='ascii')
            cox_results['interaction_summary_df'] = cph_interact.summary

            # Save interaction model summary
            interact_cox_file = os.path.join(RESULTS_OUTPUT_DIR, 'cox_model_interaction_summary.csv')
            cph_interact.summary.to_csv(interact_cox_file)
            print(f"  Interaction Cox model summary saved to {interact_cox_file}")
        else:
            print("  No interaction terms were created (possibly due to missing SDoH dummies). Skipping interaction model.")

    except Exception as cox_e:
        print(f"ERROR during Cox analysis: {cox_e}")
        traceback.print_exc()


    # --- Save Survival Analysis Results (optional pickle of summaries/plot paths) ---
    survival_results_file = os.path.join(RESULTS_OUTPUT_DIR, 'survival_analysis_results.pkl')
    survival_save_data = {'km_results': km_results, 'cox_results': cox_results}
    with open(survival_results_file, 'wb') as f:
        pickle.dump(survival_save_data, f)
    print(f"\nSurvival analysis results (plot paths, Cox summaries) saved to: {survival_results_file}")


# ==============================================================================
# === SCRIPT EXECUTION COMPLETE ================================================
# ==============================================================================
print("\n" + "="*60)
print("=== SCRIPT EXECUTION COMPLETE ===")
print("="*60)
print(f"Check the directory '{RESULTS_OUTPUT_DIR}' for analysis outputs.")

--- Initializing Setup ---
Successfully imported tableone.
Successfully imported lifelines.

Attempting to mount Google Drive...
Mounted at /content/drive
Google Drive mounted successfully.

Ensuring results directory exists: /content/drive/MyDrive/DukeDatathon_2025_Team3/AnalysisResults/
Results directory ready.

=== PART 1: DATA LOADING, PREPARATION & OUTCOME CREATION ===
Loading cohort data from: /content/drive/MyDrive/DukeDatathon_2025_Team3/Brian/files/df_cohort_v0.csv
  Initial cohort shape: (32899, 14)
Loading patients data from: /content/drive/MyDrive/DukeDatathon_2025_Team3/Brian/files/patients.csv

Converting time columns...
  Converted 'intime' in cohort.
  Converted 'deathtime' in cohort.

Merging Date of Death (dod)...
  Cohort shape after merging 'dod': (32899, 15)
  Merged 'dod' successfully.

Creating 'mortality_30day' outcome...
  Set mortality based on 'dod' within 30 days.
  Refined mortality using 'hospital_expire_flag' (added 508 potential events).
  'mortality_30d

/usr/local/lib/python3.11/dist-packages/lifelines/utils/__init__.py:1100: ConvergenceWarning: Column(s) ['insurance_OTHER'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)


  Main Effects Model Summary:
<lifelines.CoxPHFitter: fitted with 32899 total observations, 25882 right-censored observations>
             duration col = 'duration_30day'
                event col = 'mortality_30day'
                penalizer = 0.01
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 32899
number of events observed = 7017
   partial log-likelihood = -69725.850
         time fit was run = 2025-04-27 14:25:38 UTC

---
                                           coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                                                  
age                                       0.011     1.011     0.001           0.009           0.013               1.009               1.013
sofa                                      0.161     1.175     0.003           0.155           0.16

/usr/local/lib/python3.11/dist-packages/lifelines/utils/__init__.py:1100: ConvergenceWarning: Column(s) ['insurance_OTHER'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)



  Interaction Model Summary:
<lifelines.CoxPHFitter: fitted with 32899 total observations, 25882 right-censored observations>
             duration col = 'duration_30day'
                event col = 'mortality_30day'
                penalizer = 0.01
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 32899
number of events observed = 7017
   partial log-likelihood = -69693.946
         time fit was run = 2025-04-27 14:25:42 UTC

---
                                                  coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                                                         
age                                              0.011     1.011     0.001           0.009           0.013               1.009               1.014
sofa                                             0.095     1.099     0.011   

In [22]:
# Cell 6 (Revised v6): Model Comparison & Intersectional Risk (Selected Model Predictions)

# Purpose:
# 1. Compare performance (AUC) of various Logistic Regression models.
# 2. Assess risk in intersectional groups, comparing observed mortality vs.
#    average predicted risk from THREE key models:
#       - SOFA_Only
#       - SOFA_plus_Controls (SOFA + Age + Charlson)
#       - SOFA_plus_Controls_plus_SDoH_ALL (Full Model)
# 3. Generate BOTH unfiltered and filtered views of intersectional group risks.
# 4. Save key results, including model performance tables, to a specific subfolder.

print("\n" + "="*60)
print("=== PART 6: MODEL COMPARISON & INTERSECTIONAL RISK ASSESSMENT (SELECTED PREDICTIONS) ===")
print("="*60)

# --- Dependencies ---
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from IPython.display import display
import pandas as pd
import numpy as np
import os
import pickle
import traceback
import json

# --- Define Output Directory ---
# Using the directory name from your last successful run
RESULTS_OUTPUT_DIR = 'secondaryanalysis'  # Base directory as per user request
RESULTS_OUTPUT_DIR_PART6 = os.path.join(RESULTS_OUTPUT_DIR, 'Rafi_SecondAnalysis_v3')  # Specific subfolder
print(f"Ensuring specific results directory exists: {RESULTS_OUTPUT_DIR_PART6}")
try:
    os.makedirs(RESULTS_OUTPUT_DIR_PART6, exist_ok=True)
    print("Specific results directory ready.")
except Exception as e:
    print(f"ERROR creating output directory: {e}. Saving to main results dir: {RESULTS_OUTPUT_DIR}")
    RESULTS_OUTPUT_DIR_PART6 = RESULTS_OUTPUT_DIR  # Fallback

# Check if dataframe is available
if 'df_cohort' not in locals() or df_cohort is None:
    print("Skipping analysis: 'df_cohort' DataFrame not available.")
    raise SystemExit("Stopping: df_cohort required.")
else:
    print("Starting Model Comparison and Intersectional Analysis...")

    # --- Define Variables & Roles (Gender as SDoH) ---
    target = 'mortality_30day'
    sofa_col = 'sofa'
    control_cols_numeric = ['age', 'charlson_comorbidity_index']
    if 'ethnicity' in df_cohort.columns:
        race_col = 'ethnicity'
    elif 'race' in df_cohort.columns:
        race_col = 'race'
    else:
        race_col = None
    sdoh_cols = [col for col in ['gender', race_col, 'insurance', 'language'] if col is not None and col in df_cohort.columns]
    print(f"  Controls: {control_cols_numeric}")
    print(f"  SDoH Proxies: {sdoh_cols}")

    # --- Prepare Data ---
    print("\n--- Preparing Data ---")
    df_model = df_cohort.dropna(subset=[target]).copy()
    for col in sdoh_cols:
        df_model[col] = df_model[col].astype('category')
    base_numeric_cols = [sofa_col] + control_cols_numeric
    base_categorical_cols = sdoh_cols
    features_to_keep = base_numeric_cols + base_categorical_cols
    features_to_keep = [f for f in features_to_keep if f in df_model.columns]
    X = df_model[features_to_keep]
    y = df_model[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    print(f"Split data into Train ({X_train.shape[0]}) and Test ({X_test.shape[0]}).")

    # --- Evaluate Models (Function definition - simplified return) ---
    model_results = {}
    def evaluate_model(model_name, features_numeric, features_categorical, X_train_func, y_train_func, X_test_func, y_test_func):
        print(f"\n--- Evaluating Model: {model_name} ---")
        current_features = features_numeric + features_categorical
        X_train_subset = X_train_func[[f for f in current_features if f in X_train_func.columns]]
        X_test_subset = X_test_func[[f for f in current_features if f in X_test_func.columns]]
        print(f"  Using {X_train_subset.shape[1]} features: {X_train_subset.columns.tolist()}")
        transformers = []
        num_in_subset = [f for f in features_numeric if f in X_train_subset.columns]
        cat_in_subset = [f for f in features_categorical if f in X_train_subset.columns]
        if num_in_subset:
            transformers.append(('num', StandardScaler(), num_in_subset))
        if cat_in_subset:
            transformers.append(('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first'), cat_in_subset))
        if not transformers:
            print("  ERROR: No valid features. Skipping.")
            return None
        pipeline = Pipeline(steps=[
            ('preprocessor', ColumnTransformer(transformers=transformers, remainder='passthrough')),
            ('classifier', LogisticRegression(solver='liblinear', random_state=42, class_weight='balanced', max_iter=1000))
        ])
        try:
            pipeline.fit(X_train_subset, y_train_func)
            print("  Model trained successfully.")
            y_pred_proba = pipeline.predict_proba(X_test_subset)[:, 1]
            auc = roc_auc_score(y_test_func, y_pred_proba)
            print(f"  Test Set AUC: {auc:.4f}")
            # Store only model and AUC for this part
            return {'model': pipeline, 'auc': auc}
        except Exception as e:
            print(f"  ERROR evaluating {model_name}: {e}")
            traceback.print_exc()
            return None

    # --- 1. Evaluate Key Model Configurations ---
    print("\n" + "="*30)
    print("=== Evaluating Key Model Performance ===")
    print("="*30)
    model_results['SOFA_Only'] = evaluate_model('SOFA_Only', [sofa_col], [], X_train, y_train, X_test, y_test)
    model_results['SOFA_plus_Controls'] = evaluate_model('SOFA_plus_Controls', [sofa_col] + control_cols_numeric, [], X_train, y_train, X_test, y_test)
    model_results['SOFA_plus_Controls_plus_SDoH_ALL'] = evaluate_model('SOFA_plus_Controls_plus_SDoH_ALL', [sofa_col] + control_cols_numeric, sdoh_cols, X_train, y_train, X_test, y_test)

    # --- Summarize & Save Model Comparison ---
    print("\n" + "="*30)
    print("=== Model Performance Summary (Test Set AUC) ===")
    print("="*30)
    auc_summary = {name: res['auc'] for name, res in model_results.items() if res}
    auc_series = pd.Series(auc_summary).sort_values(ascending=False)
    print(auc_series.to_string(float_format='{:.4f}'.format))
    model_comp_file = os.path.join(RESULTS_OUTPUT_DIR_PART6, 'model_performance_comparison_AUC_selected.csv')
    auc_series.to_frame(name='Test AUC').to_csv(model_comp_file)
    print(f"\nModel AUC comparison saved to: {model_comp_file}")

    # --- 2. Re-examine Intersectional Groups with Selected Model Predictions ---
    print("\n" + "="*30)
    print("=== Intersectional Group Risk Assessment ===")
    print("="*30)
    # Define age group column name
    age_group_col_intersect = 'age_group'
    if age_group_col_intersect not in df_model.columns:
        try:
            bins = [0, 49, 64, 79, np.inf]
            labels = ['<=49', '50-64', '65-79', '80+']
            df_model[age_group_col_intersect] = pd.cut(df_model['age'], bins=bins, labels=labels, right=True)
        except:
            age_group_col_intersect = None

    intersection_factors_intersect = [f for f in [age_group_col_intersect, 'gender', race_col, 'insurance'] if f is not None and f in df_model.columns]

    if not intersection_factors_intersect:
        print("ERROR: Not enough factors for intersectional analysis.")
    else:
        print(f"Grouping by: {intersection_factors_intersect}")
        grouped_intersect = df_model.groupby(intersection_factors_intersect, observed=False)
        intersectional_stats = grouped_intersect[target].agg(['size', 'mean']).rename(columns={'size': 'Group Size', 'mean': 'Observed Mortality Rate'})

        # --- Add predictions from the THREE key models ---
        models_to_predict = {
            'SOFA_Only': [sofa_col],
            'SOFA_plus_Controls': [sofa_col] + control_cols_numeric,
            'SOFA_plus_Controls_plus_SDoH_ALL': [sofa_col] + control_cols_numeric + sdoh_cols
        }

        for model_key, features_used in models_to_predict.items():
            if model_key in model_results and model_results[model_key]:
                print(f"Adding predicted risk from model: {model_key}...")
                try:
                    pipeline = model_results[model_key]['model']
                    # Select the exact features used by THIS model
                    current_features = [f for f in features_used if f in df_model.columns]
                    X_predict = df_model[current_features]
                    pred_col_name = f'pred_risk_{model_key}'
                    df_model[pred_col_name] = pipeline.predict_proba(X_predict)[:, 1]
                    # Calculate average prediction for each intersectional group
                    avg_predicted_risk = df_model.groupby(intersection_factors_intersect, observed=False)[pred_col_name].mean()
                    intersectional_stats = intersectional_stats.join(avg_predicted_risk)
                    # Rename column for clarity
                    intersectional_stats.rename(columns={pred_col_name: f'Avg Pred Risk ({model_key})'}, inplace=True)
                except Exception as pred_e:
                    print(f"  ERROR predicting with {model_key} model: {pred_e}")
            else:
                print(f"Model '{model_key}' not found or failed training. Skipping predictions.")

        # Sort by observed mortality rate
        all_groups_sorted = intersectional_stats.sort_values(by='Observed Mortality Rate', ascending=False).copy()

        # Format percentages for display
        all_groups_sorted['Observed Mortality (%)'] = (all_groups_sorted['Observed Mortality Rate'] * 100).map('{:.1f}%'.format)
        pred_cols_to_format = [col for col in all_groups_sorted.columns if col.startswith('Avg Pred Risk')]
        for pred_col in pred_cols_to_format:
            all_groups_sorted[f"{pred_col} (%)"] = (all_groups_sorted[pred_col] * 100).map('{:.1f}%'.format)

        # Define display columns dynamically based on successful predictions
        display_cols_base = ['Group Size', 'Observed Mortality (%)']
        display_cols_preds = [f"{col} (%)" for col in pred_cols_to_format]
        display_cols_combined = display_cols_base + display_cols_preds

        # --- Display & Save UNFILTERED Results ---
        N_TOP_DISPLAY = 15
        print(f"\n--- Top {N_TOP_DISPLAY} Groups by Observed Mortality (ALL SIZES) ---")
        print("!!! WARNING: Rates/Predictions for groups with small 'Group Size' are UNRELIABLE !!!")
        display(all_groups_sorted[[c for c in display_cols_combined if c in all_groups_sorted.columns]].head(N_TOP_DISPLAY))
        intersectional_file_full = os.path.join(RESULTS_OUTPUT_DIR_PART6, 'intersectional_stats_multimodel_preds_full.csv')
        all_groups_sorted.to_csv(intersectional_file_full)
        print(f"\nFull intersectional stats saved to: {intersectional_file_full}")

        print("\n" + "-"*50)

        # --- Filter by Minimum Group Size & Display/Save ---
        MINIMUM_GROUP_SIZE_THRESHOLD = 30  # ADJUSTABLE threshold
        print(f"Filtering groups to include only those with >= {MINIMUM_GROUP_SIZE_THRESHOLD} patients...")
        reliable_groups_sorted = all_groups_sorted[all_groups_sorted['Group Size'] >= MINIMUM_GROUP_SIZE_THRESHOLD].copy()

        if reliable_groups_sorted.empty:
            print(f"No groups found with at least {MINIMUM_GROUP_SIZE_THRESHOLD} patients.")
        else:
            print(f"\n--- Top {N_TOP_DISPLAY} Groups by Observed Mortality (Min Size: {MINIMUM_GROUP_SIZE_THRESHOLD}) ---")
            display(reliable_groups_sorted[[c for c in display_cols_combined if c in reliable_groups_sorted.columns]].head(N_TOP_DISPLAY))
            intersectional_results_filtered_file = os.path.join(RESULTS_OUTPUT_DIR_PART6, f'intersectional_stats_multimodel_preds_filtered_MIN{MINIMUM_GROUP_SIZE_THRESHOLD}.csv')
            reliable_groups_sorted.to_csv(intersectional_results_filtered_file)
            print(f"\nFiltered results (Min size {MINIMUM_GROUP_SIZE_THRESHOLD}) saved to: {intersectional_results_filtered_file}")

    # Save only the key trained model pipelines needed for intersectional prediction
    models_to_save = {}
    for key in ['SOFA_Only', 'SOFA_plus_Controls', 'SOFA_plus_Controls_plus_SDoH_ALL']:
        if key in model_results and model_results[key]:
            models_to_save[key] = model_results[key]['model']

    if models_to_save:
        models_file = os.path.join(RESULTS_OUTPUT_DIR_PART6, 'key_logistic_models.pkl')
        try:
            with open(models_file, 'wb') as f:
                pickle.dump(models_to_save, f)
            print(f"\nKey trained model pipelines saved to: {models_file}")
        except Exception as save_e:
            print(f"Error saving key model pipelines: {save_e}")

print("\nModel Comparison & Intersectional Risk Assessment Complete.")


=== PART 6: MODEL COMPARISON & INTERSECTIONAL RISK ASSESSMENT (SELECTED PREDICTIONS) ===
Ensuring specific results directory exists: secondaryanalysis/Rafi_SecondAnalysis_v3
Specific results directory ready.
Starting Model Comparison and Intersectional Analysis...
  Controls: ['age', 'charlson_comorbidity_index']
  SDoH Proxies: ['gender', 'ethnicity', 'insurance', 'language']

--- Preparing Data ---
Split data into Train (24674) and Test (8225).

=== Evaluating Key Model Performance ===

--- Evaluating Model: SOFA_Only ---
  Using 1 features: ['sofa']
  Model trained successfully.
  Test Set AUC: 0.6916

--- Evaluating Model: SOFA_plus_Controls ---
  Using 3 features: ['sofa', 'age', 'charlson_comorbidity_index']
  Model trained successfully.
  Test Set AUC: 0.7482

--- Evaluating Model: SOFA_plus_Controls_plus_SDoH_ALL ---
  Using 7 features: ['sofa', 'age', 'charlson_comorbidity_index', 'gender', 'ethnicity', 'insurance', 'language']
  Model trained successfully.
  Test Set AUC: 0.

Group Size  \
age_group gender ethnicity              insurance               
<=49      M      ASIAN - CHINESE        Medicare            2   
80+       F      WHITE - RUSSIAN        Other               1   
<=49      F      WHITE - RUSSIAN        Medicaid            1   
65-79     F      WHITE                  OTHER               1   
80+       M      WHITE - OTHER EUROPEAN Private             1   
50-64     F      UNABLE TO OBTAIN       UNKNOWN             1   
65-79     M      UNABLE TO OBTAIN       Other               1   
80+       F      BLACK/AFRICAN AMERICAN Other               1   
          M      UNABLE TO OBTAIN       UNKNOWN             1   
50-64     M      UNABLE TO OBTAIN       Other               2   
65-79     F      WHITE - RUSSIAN        Private             1   
80+       F      UNKNOWN                UNKNOWN             3   
                                        Other               1   
65-79     F      UNKNOWN                UNKNOWN             8   
50-64     F      UNKNOWN                UNKNOWN            12   

                                                  Observed Mortality (%)  \
age_group gender ethnicity              insurance                          
<=49      M      ASIAN - CHINESE        Medicare                  100.0%   
80+       F      WHITE - RUSSIAN        Other                     100.0%   
<=49      F      WHITE - RUSSIAN        Medicaid                  100.0%   
65-79     F      WHITE                  OTHER                     100.0%   
80+       M      WHITE - OTHER EUROPEAN Private                   100.0%   
50-64     F      UNABLE TO OBTAIN       UNKNOWN                   100.0%   
65-79     M      UNABLE TO OBTAIN       Other                     100.0%   
80+       F      BLACK/AFRICAN AMERICAN Other                     100.0%   
          M      UNABLE TO OBTAIN       UNKNOWN                   100.0%   
50-64     M      UNABLE TO OBTAIN       Other                     100.0%   
65-79     F      WHITE - RUSSIAN        Private                   100.0%   
80+       F      UNKNOWN                UNKNOWN                   100.0%   
                                        Other                     100.0%   
65-79     F      UNKNOWN                UNKNOWN                    87.5%   
50-64     F      UNKNOWN                UNKNOWN                    83.3%   

                                                  Avg Pred Risk (SOFA_Only) (%)  \
age_group gender ethnicity              insurance                                 
<=49      M      ASIAN - CHINESE        Medicare                          47.0%   
80+       F      WHITE - RUSSIAN        Other                             86.6%   
<=49      F      WHITE - RUSSIAN        Medicaid                          37.3%   
65-79     F      WHITE                  OTHER                             32.8%   
80+       M      WHITE - OTHER EUROPEAN Private                           47.0%   
50-64     F      UNABLE TO OBTAIN       UNKNOWN                           56.9%   
65-79     M      UNABLE TO OBTAIN       Other                             52.0%   
80+       F      BLACK/AFRICAN AMERICAN Other                             42.1%   
          M      UNABLE TO OBTAIN       UNKNOWN                           78.1%   
50-64     M      UNABLE TO OBTAIN       Other                             72.9%   
65-79     F      WHITE - RUSSIAN        Private                           56.9%   
80+       F      UNKNOWN                UNKNOWN                           67.5%   
                                        Other                             78.1%   
65-79     F      UNKNOWN                UNKNOWN                           48.3%   
50-64     F      UNKNOWN                UNKNOWN                           56.6%   

                                                  Avg Pred Risk (SOFA_plus_Controls) (%)  \
age_group gender ethnicity              insurance                                          
<=49      M      ASIAN - CHINESE        Medicare        


Full intersectional stats saved to: secondaryanalysis/Rafi_SecondAnalysis_v3/intersectional_stats_multimodel_preds_full.csv

--------------------------------------------------
Filtering groups to include only those with >= 30 patients...

--- Top 15 Groups by Observed Mortality (Min Size: 30) ---


Group Size  \
age_group gender ethnicity              insurance               
80+       F      UNKNOWN                Medicare          266   
<=49      M      UNKNOWN                UNKNOWN            42   
80+       M      UNABLE TO OBTAIN       Medicare           43   
50-64     F      UNKNOWN                Medicaid           70   
80+       F      UNABLE TO OBTAIN       Medicare           45   
                 WHITE - RUSSIAN        Medicare          102   
          M      UNKNOWN                Medicare          262   
65-79     F      UNKNOWN                Medicare          339   
80+       F      BLACK/AFRICAN AMERICAN Medicaid           30   
50-64     M      UNKNOWN                Medicaid          137   
80+       F      WHITE - OTHER EUROPEAN Medicare           59   
          M      WHITE - RUSSIAN        Medicare           93   
65-79     M      UNKNOWN                Medicaid           30   
80+       F      BLACK/AFRICAN AMERICAN Medicare          291   
65-79     F      UNKNOWN                Private            42   

                                                  Observed Mortality (%)  \
age_group gender ethnicity              insurance                          
80+       F      UNKNOWN                Medicare                   53.8%   
<=49      M      UNKNOWN                UNKNOWN                    47.6%   
80+       M      UNABLE TO OBTAIN       Medicare                   44.2%   
50-64     F      UNKNOWN                Medicaid                   40.0%   
80+       F      UNABLE TO OBTAIN       Medicare                   40.0%   
                 WHITE - RUSSIAN        Medicare                   38.2%   
          M      UNKNOWN                Medicare                   37.8%   
65-79     F      UNKNOWN                Medicare                   37.2%   
80+       F      BLACK/AFRICAN AMERICAN Medicaid                   36.7%   
50-64     M      UNKNOWN                Medicaid                   36.5%   
80+       F      WHITE - OTHER EUROPEAN Medicare                   35.6%   
          M      WHITE - RUSSIAN        Medicare                   35.5%   
65-79     M      UNKNOWN                Medicaid                   33.3%   
80+       F      BLACK/AFRICAN AMERICAN Medicare                   33.3%   
65-79     F      UNKNOWN                Private                    33.3%   

                                                  Avg Pred Risk (SOFA_Only) (%)  \
age_group gender ethnicity              insurance                                 
80+       F      UNKNOWN                Medicare                          50.3%   
<=49      M      UNKNOWN                UNKNOWN                           48.9%   
80+       M      UNABLE TO OBTAIN       Medicare                          49.5%   
50-64     F      UNKNOWN                Medicaid                          56.0%   
80+       F      UNABLE TO OBTAIN       Medicare                          44.4%   
                 WHITE - RUSSIAN        Medicare                          48.4%   
          M      UNKNOWN                Medicare                          50.4%   
65-79     F      UNKNOWN                Medicare                          47.6%   
80+       F      BLACK/AFRICAN AMERICAN Medicaid                          43.8%   
50-64     M      UNKNOWN                Medicaid                          55.0%   
80+       F      WHITE - OTHER EUROPEAN Medicare                          45.6%   
          M      WHITE - RUSSIAN        Medicare                          50.1%   
65-79     M      UNKNOWN                Medicaid                          55.8%   
80+       F      BLACK/AFRICAN AMERICAN Medicare                          47.5%   
65-79     F      UNKNOWN                Private                           45.4%   

                                                  Avg Pred Risk (SOFA_plus_Controls) (%)  \
age_group gender ethnicity              insurance                                          
80+       F      UNKNOWN                Medicare        


Filtered results (Min size 30) saved to: secondaryanalysis/Rafi_SecondAnalysis_v3/intersectional_stats_multimodel_preds_filtered_MIN30.csv

Key trained model pipelines saved to: secondaryanalysis/Rafi_SecondAnalysis_v3/key_logistic_models.pkl

Model Comparison & Intersectional Risk Assessment Complete.
